In [3]:
import json
import torch
from PIL import Image, ImageTk
import tkinter as tk
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.auth.exceptions import RefreshError
import requests
from io import BytesIO
import hashlib

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
lemmatizer = WordNetLemmatizer()

cache_imagenes = {}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using cache found in C:\Users\andre/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-4-12 Python-3.10.4 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [5]:
def filtrar_stopwords(texto):
    """Eliminar las stopwords del texto."""
    stopwords_english = set(stopwords.words('english'))
    conceptos_clave = [word for word in texto.split() if word.lower() not in stopwords_english]
    return conceptos_clave

def procesar_imagenes(consulta):
    """Procesar las imágenes coincidentes con la consulta utilizando la caché."""
    conceptos_clave = filtrar_stopwords(consulta)
    conceptos_clave = [lemmatizer.lemmatize(concepto) for concepto in conceptos_clave]
    imagenes_coincidentes = []

    for _, (imagen, nombres_clases) in cache_imagenes.items():
        if all(concepto in nombres_clases for concepto in conceptos_clave):
            imagenes_coincidentes.append(imagen)
    
    return imagenes_coincidentes

def obtener_nombres_clases(results):
    """Obtener los nombres de las clases detectadas en los resultados."""
    nombres_clases = []
    for tensor_pred in results.pred:
        indices_clases = tensor_pred[:, -1].int()
        nombres_clases.extend([results.names[int(idx)] for idx in indices_clases])
    return nombres_clases

def mostrar_imagenes(imagenes):
    """Mostrar las imágenes coincidentes en la interfaz gráfica."""
    for widget in frame_imagenes.winfo_children():
        widget.destroy()
    num_columnas = 4
    for idx, imagen in enumerate(imagenes):
        fila = idx // num_columnas
        columna = idx % num_columnas
        ancho, alto = imagen.size
        aspect_ratio = ancho / alto
        nuevo_ancho = 200
        nuevo_alto = int(nuevo_ancho / aspect_ratio)
        imagen_redimensionada = imagen.resize((nuevo_ancho, nuevo_alto), Image.LANCZOS)
        imagen_tk = ImageTk.PhotoImage(imagen_redimensionada)
        label_imagen = tk.Label(frame_imagenes, image=imagen_tk)
        label_imagen.image = imagen_tk
        label_imagen.grid(row=fila, column=columna, padx=5, pady=5)

def procesar_consulta():
    """Procesar la consulta ingresada por el usuario."""
    consulta = entry_consulta.get()
    imagenes_coincidentes = procesar_imagenes(consulta)
    mostrar_imagenes(imagenes_coincidentes)

def obtener_imagenes_google_photos():
    """Obtener las imágenes de Google Photos."""
    SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']

    # Leer la configuración desde el archivo config.json
    with open('config.json') as config_file:
        config = json.load(config_file)
    client_secret_file = config['client_secret_file']

    flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
    credentials = flow.run_local_server()

    try:
        credentials.refresh(Request())
    except RefreshError as e:
        print("Error al refrescar los tokens:", e)
        return []

    url = 'https://photoslibrary.googleapis.com/v1/mediaItems'
    headers = {
        'Authorization': f'Bearer {credentials.token}'
    }

    imagenes = []
    next_page_token = None
    while True:
        params = {'pageSize': '10'}
        if next_page_token:
            params['pageToken'] = next_page_token

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            for item in data.get('mediaItems', []):
                photo_url = item['baseUrl'] + '=d'
                response_photo = requests.get(photo_url)
                if response_photo.status_code == 200:
                    imagenes.append(response_photo.content)
                else:
                    print(f"Error al obtener la imagen de {photo_url}.")
            next_page_token = data.get('nextPageToken')
            if not next_page_token:
                break
        else:
            print(f"Error al hacer la solicitud: {response.status_code}, {response.text}")
            break

    return imagenes

def cachear_imagenes(imagenes):
    """Procesar y cachear todas las imágenes inicialmente."""
    for imagen in imagenes:
        img_bytes = imagen.tobytes()
        img_hash = hashlib.md5(img_bytes).hexdigest()
        if img_hash not in cache_imagenes:
            results = model(imagen)
            nombres_clases = obtener_nombres_clases(results)
            cache_imagenes[img_hash] = (imagen, nombres_clases)




In [6]:
# Descargar las imágenes de Google Photos
imagenes_google = obtener_imagenes_google_photos()
imagenes_google_pil = [Image.open(BytesIO(img)) for img in imagenes_google]

# Cachear todas las imágenes descargadas
cachear_imagenes(imagenes_google_pil)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=888929256035-s07vr96dinqchvpuptrm4m3a6l0n2tco.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fphotoslibrary.readonly&state=64SgLYbUWU3i9tRb71F1hfVMJ1avql&access_type=offline


In [7]:
# Crear la ventana principal
root = tk.Tk()
root.title("Búsqueda de imágenes")

# Crear un marco para la entrada de consulta
frame_consulta = tk.Frame(root)
frame_consulta.pack(pady=10)

# Etiqueta de entrada de consulta
label_consulta = tk.Label(frame_consulta, text="Ingrese su consulta:")
label_consulta.grid(row=0, column=0)

# Entrada de consulta
entry_consulta = tk.Entry(frame_consulta, width=50)
entry_consulta.grid(row=0, column=1)

# Botón de búsqueda
button_buscar = tk.Button(frame_consulta, text="Buscar", command=procesar_consulta)
button_buscar.grid(row=0, column=2, padx=10)

# Marco para mostrar las imágenes
frame_imagenes = tk.Frame(root)
frame_imagenes.pack(padx=10, pady=5)

# Ejecutar la aplicación
root.mainloop()